In [101]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial
import random
import os
from girth import twopl_mml,twopl_jml

In [102]:
test_questions=pd.read_csv("General_Knowledge_Better_Prompt.csv")
test_questions['question_id']='Q'+test_questions['question_id'].astype(str)
test_questions=test_questions.iloc[41:47]
test_questions.reset_index(drop=True,inplace=True)

In [103]:
column_names=list(test_questions['question_id'])
responses=pd.DataFrame(columns=column_names)

In [104]:
## User Testing Class
class TestTaker:
    def __init__(self, questions):
        self.questions = questions
        self.responses = {}
        self.current_question = 0
        self.name = ""
        self.response_df=pd.DataFrame()
    def ask_question(self):
        if self.current_question < len(self.questions):
            clear_output(wait=True)
            question = self.questions.iloc[self.current_question]
            print(f"Question {self.current_question + 1}: {question['Question']}")
            options_buttons=[]
            options=['Option A','Option B','Option C', 'Option D']
            for idx, option in enumerate(options):
                button = widgets.Button(description=f"{chr(65+idx)}: {question[option]}")
                button.on_click(partial(self.record_response, question=question, option=option))
                button.layout.width = 'auto'
                options_buttons.append(button)

            display(widgets.VBox(options_buttons))
        else:
            self.complete_test()

    def record_response(self, button, question, option):
        correct_option_letter = question['Correct Option'][-1]
        self.responses[question['question_id']] = 1 if option == question['Correct Option'] else 0
        self.current_question += 1
        self.ask_question()

    def start_test(self, name):
        self.name = name
        self.ask_question()

    def complete_test(self):
        global responses
        clear_output(wait=True)
        print(f"Test completed for {self.name}!")
        self.name
        row_index = [self.name]  
        self.response_df = pd.DataFrame(self.responses,index=row_index)  
        responses=pd.concat([responses,self.response_df],axis=0)
        return self.response_df


In [107]:
test=TestTaker(test_questions)
test.start_test(name='Amit')


Test completed for Amit!


In [108]:
responses

,Q41,Q42,Q43,Q44,Q45,Q46
Sarath,0,1,0,0,0,0
Amit,0,0,0,0,0,0


In [88]:
responses.to_csv('Responses.csv')

In [109]:
import numpy as np
from girth import twopl_mml,twopl_jml

response_matrix=responses.to_numpy().transpose().astype(int)
estimates = twopl_mml(response_matrix)

discrimination_estimates = estimates['Discrimination']
difficulty_estimates = estimates['Difficulty']